In [12]:
import numpy as np
import pandas as pd
import json
import os
from pathlib import Path

In [13]:
directory = Path(r"C:\Users\johnkerf\Desktop\Quantum-Computing\Quantum-Computing\SUSY\Wess-Zumino\VQE\DE\Files")
json_files = list(directory.rglob("*.json"))

In [17]:
converged_runs = []
mean_iters = []
min_e = []
delta_min_e = []
median_e = []
delta_median_e = []
exact_e = []
parallel_run_time = []
total_VQE_time = []
total_device_time = []
BCs = []
potential = []
Cs = []
sites = []
cutoffs =[]
percentages= []

for file in json_files:

    with open(file, 'r') as file:
        data = json.load(file)

    BCs.append(data['boundary_condition'])
    potential.append(data['potential'])
    Cs.append(data['c'])
    sites.append(data['N'])
    cutoffs.append(data['cutoff'])

    exact_min_e = np.min(data['exact_eigenvalues'])
    exact_e.append(exact_min_e)
    
    converged_runs.append(sum(data['success']))
    mean_iters.append(int(np.round(np.mean(data['num_iters']))))

    min_e.append(np.min(data['results']))
    delta_min_e.append(abs(exact_min_e - np.min(data['results'])))

    median_e.append(np.median(data['results']))   
    delta_median_e.append(abs(exact_min_e - np.median(data['results']))) 

    total_VQE_time.append(data['total_VQE_time'])
    parallel_run_time.append(data['parallel_run_time'])
    total_device_time.append(data['total_device_time'])

    dev_perc = (pd.Timedelta(data['total_device_time']).total_seconds() / pd.Timedelta(data['total_VQE_time']).total_seconds()) * 100
    percentages.append(dev_perc)

    

In [ ]:
df = pd.DataFrame({
    'boundary condition': BCs,
    'Potential': potential,
    'C': Cs,
    'N sites': sites,
    'cutoff': cutoffs,
    'Converged Runs': [f"{c}/100" for c in converged_runs],
    'Mean Iter': mean_iters,
    'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    'Exact': [f"{num:.5e}" for num in exact_e],
    'total_VQE_time': total_VQE_time,
    'parallel_run_time': parallel_run_time,
    'total_device_time': total_device_time,
    'percentage_device_time': percentages
})

In [20]:
df[df['boundary condition'] == 'dirichlet']

,boundary condition,Potential,C,N sites,cutoff,Converged Runs,Mean Iter,VQE min E,Delta min E,VQE median E,Delta median E,Exact,total_VQE_time,parallel_run_time,total_device_time,percentage_device_time
0,dirichlet,linear,NaN,2,2,100/100,309,6.96601e-03,1.56125e-17,6.96601e-03,1.56125e-17,6.96601e-03,0:55:48.652366,0:01:50.861943,0:01:42.959026,3.074641
1,dirichlet,linear,NaN,2,4,100/100,1723,3.22506e-05,1.24838e-15,3.22506e-05,1.24838e-15,3.22506e-05,8:12:18.020585,0:16:13.807416,0:10:37.182241,2.157160
2,dirichlet,linear,NaN,2,8,100/100,3835,4.54071e-10,2.01469e-15,4.54071e-10,2.01469e-15,4.54073e-10,"3 days, 22:07:13.755853",3:01:13.223567,0:33:39.308704,0.595959
3,dirichlet,linear,NaN,3,2,100/100,1846,-9.97449e-02,1.77636e-15,-9.97449e-02,1.87350e-15,-9.97449e-02,8:47:39.945892,0:20:51.565804,0:11:23.976230,2.160383
4,dirichlet,quadratic,-0.2,2,2,100/100,718,-9.11346e-01,4.44089e-16,-9.11346e-01,4.44089e-16,-9.11346e-01,2:07:14.381480,0:04:06.040245,0:03:50.435198,3.018387
5,dirichlet,quadratic,-0.2,2,4,0/100,10000,2.74984e-01,9.30217e-02,2.86815e-01,1.04853e-01,1.81963e-01,"1 day, 23:11:39.872493",1:22:54.638178,1:00:22.217078,2.131972
6,dirichlet,quadratic,-0.2,2,8,0/100,10000,1.72118e-01,4.10838e-02,3.07812e-01,1.76778e-01,1.31034e-01,"10 days, 13:21:28.467071",7:19:38.824115,1:31:29.206250,0.601828
7,dirichlet,quadratic,-0.2,3,2,83/100,5913,-1.27825e+00,1.77636e-15,-1.27825e+00,1.33227e-15,-1.27825e+00,"1 day, 4:09:01.551366",0:54:24.371342,0:36:02.727858,2.134098
8,dirichlet,quadratic,-0.8,2,2,100/100,1296,-9.11346e-01,4.44089e-16,-9.11346e-01,4.44089e-16,-9.11346e-01,3:47:46.852191,0:08:08.926290,0:06:53.016300,3.022029
9,dirichlet,quadratic,-0.8,2,4,82/100,7094,-1.15143e+00,8.43769e-15,-1.15143e+00,8.43769e-15,-1.15143e+00,"1 day, 9:55:26.357553",1:02:05.550959,0:43:40.293509,2.145559


In [21]:
df[df['boundary condition'] == 'periodic']

,boundary condition,Potential,C,N sites,cutoff,Converged Runs,Mean Iter,VQE min E,Delta min E,VQE median E,Delta median E,Exact,total_VQE_time,parallel_run_time,total_device_time,percentage_device_time
12,periodic,linear,NaN,2,2,100/100,132,-2.22045e-16,0.00000e+00,-2.22045e-16,0.00000e+00,-2.22045e-16,0:24:56.847013,0:00:47.310164,0:00:45.433164,3.035258
13,periodic,linear,NaN,2,4,100/100,1667,-2.22045e-16,0.00000e+00,-2.22045e-16,0.00000e+00,-2.22045e-16,7:56:18.474934,0:17:12.887875,0:10:20.086215,2.169767
14,periodic,linear,NaN,2,8,99/100,3759,-2.22045e-16,0.00000e+00,-2.22045e-16,0.00000e+00,-2.22045e-16,"3 days, 21:15:52.648715",2:46:11.110921,0:33:26.158677,0.597511
15,periodic,linear,NaN,3,2,99/100,3801,-2.80776e-01,1.11022e-16,-2.79617e-01,1.15987e-03,-2.80776e-01,18:01:03.908160,0:38:48.539556,0:23:24.668551,2.165563
16,periodic,quadratic,-0.2,2,2,99/100,1700,-8.24214e-01,0.00000e+00,-8.24214e-01,0.00000e+00,-8.24214e-01,4:56:01.564478,0:17:19.180193,0:08:50.851884,2.988768
17,periodic,quadratic,-0.2,2,4,86/100,6948,4.06210e-01,1.77636e-15,4.06210e-01,2.10942e-15,4.06210e-01,"1 day, 9:06:42.645901",1:02:09.455642,0:42:51.719261,2.157435
18,periodic,quadratic,-0.2,2,8,0/100,10000,3.46274e-01,1.21569e-14,3.59641e-01,1.33669e-02,3.46274e-01,"10 days, 6:49:59.109461",7:13:19.719472,1:29:28.312910,0.604132
19,periodic,quadratic,-0.2,3,2,97/100,4510,-1.43423e+00,1.06581e-14,-1.43423e+00,1.06581e-14,-1.43423e+00,21:36:59.840256,0:42:03.071595,0:27:52.564425,2.149278
20,periodic,quadratic,-0.8,2,2,100/100,1777,-8.24214e-01,4.44089e-16,-8.24214e-01,6.66134e-16,-8.24214e-01,5:09:08.024983,0:10:53.246112,0:09:20.862905,3.023842
21,periodic,quadratic,-0.8,2,4,98/100,3755,3.57184e-01,3.88578e-16,3.57184e-01,4.44089e-16,3.57184e-01,17:46:30.340237,0:37:01.001464,0:23:00.724212,2.157707
